In [95]:
%reset-f
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
from res_ind_lib import *

from sorted_nicely import *
from replace_with_warning import *

import os, time
import warnings
warnings.filterwarnings("always",category=UserWarning)

In [97]:
import numpy as np
import pandas as pd

#Pandas display options
# pd.set_option('display.max_colwidth', 200)
# pd.set_option('display.width', 200)
# pd.set_option('display.precision', 10)
# pd.set_option('display.max_rows', 500)

import matplotlib.pylab as plt
%matplotlib inline
font = {'family' : 'serif',
    'weight' : 'normal',
    'size'   : 15}
plt.rc('font', **font)

# combining all inputs in one dataframe

In [98]:
economy = "country" #province, deparmtent
event_level = [economy, "hazard", "rp"]

## macro

In [99]:
macro= pd.read_csv("intermediate/macro.csv", index_col=economy).dropna()

df=macro.copy()
df.columns = ["macro_"+c for c in df]

df.sample(n=2)

,macro_gdp_pc_pp,macro_pop,macro_urbanization_rate,macro_prepare_scaleup,macro_borrow_abi,macro_avg_prod_k,macro_T_rebuild_K,macro_pi,macro_income_elast,macro_rho,macro_shareable,macro_max_increased_spending,macro_fa_glofris,macro_protection,macro_tau_tax
country,,,,,,,,,,,,,,,
Romania,18184.109733,19981358.0,0.54564,0.6,0.691667,0.267748,3,0.2,1.5,0.06,0.8,0.05,0.073684,1.0,0.423380
Paraguay,7833.074002,6802295.0,0.59666,0.8,0.666667,0.368273,3,0.2,1.5,0.06,0.8,0.05,0.027504,1.0,0.184327


## cat_info

In [100]:
cat_info=pd.read_csv("intermediate/cat_info.csv", index_col=[economy, "income_cat"])

hop=cat_info.copy()

hop=hop.unstack("income_cat").rename(
    columns={"nonpoor":"_cat_info__nonpoor", "poor":"_cat_info__poor"})


hop.columns=[''.join(col) for col in hop.columns.values]

df[hop.columns]=hop

df.head()



,macro_gdp_pc_pp,macro_pop,macro_urbanization_rate,macro_prepare_scaleup,macro_borrow_abi,macro_avg_prod_k,macro_T_rebuild_K,macro_pi,macro_income_elast,macro_rho,...,gamma_SP_cat_info__nonpoor,gamma_SP_cat_info__poor,k_cat_info__nonpoor,k_cat_info__poor,fa_cat_info__nonpoor,fa_cat_info__poor,v_cat_info__nonpoor,v_cat_info__poor,shew_cat_info__nonpoor,shew_cat_info__poor
country,,,,,,,,,,,,,,,,,,,,,
Albania,9960.940726,2897366.0,0.57407,0.000000,0.175000,0.185280,3,0.2,1.5,0.06,...,1.094243,0.623028,61811.780436,21560.668539,0.022104,0.021223,0.351500,0.7,0.00,0.00
Angola,7488.011760,21471618.0,0.44050,0.000000,0.175000,0.432397,3,0.2,1.5,0.06,...,1.160476,0.358097,20523.147061,4494.564269,0.002814,0.012449,0.600000,0.7,0.00,0.00
Argentina,18087.196262,41446246.0,0.91751,0.733333,0.375000,0.269550,3,0.2,1.5,0.06,...,1.171127,0.315492,80282.025353,14379.760092,0.008573,0.008979,0.397254,0.7,0.64,0.64
Armenia,7526.836419,2976566.0,0.62673,0.400000,0.425000,0.366696,3,0.2,1.5,0.06,...,1.043768,0.824929,24135.209063,6089.709776,0.053080,0.052914,0.237500,0.3,0.32,0.32
Australia,42834.016760,23129300.0,0.89423,0.800000,0.916667,0.273602,3,0.2,1.5,0.06,...,0.928076,1.287696,188801.470089,27572.673316,0.003927,0.004331,0.538600,0.7,0.80,0.80


## hazard

In [101]:
hazard_ratios = pd.read_csv("intermediate/hazard_ratios.csv", index_col=event_level+["income_cat"]).dropna()
hazard_ratios.sample(n=2)

,,,,fa,shew
country,hazard,rp,income_cat,,
Gabon,earthquake,1000.0,nonpoor,0.007522,0.0
Comoros,wind,1.0,poor,0.000124,0.6


### shew

In [102]:
df["shew_for_hazard_ratio"]= average_over_rp(hazard_ratios).shew.max(level="country")

### single fa per hazard

In [103]:
hop = average_over_rp(hazard_ratios).fa.unstack().nonpoor.unstack().rename(columns=lambda c:"hazard_ratio_fa__"+c)
hop.head()

# hop=hop.drop(["hazard_ratio_fa__surge"],axis=1)

df[hop.columns]    =hop

df.head().T;

hop;

### different exposure for poor to floods

In [104]:
df["hazard_ratio_flood_poor"] = average_over_rp(hazard_ratios).fa.unstack().unstack().poor.flood
# .assign(bias=lambda x:x.poor/x.nonpoor).bias.replace(1,np.nan).median(level="country")

In [105]:

paf=average_over_rp(hazard_ratios).fa.unstack().unstack().swaplevel(0,1,axis=1)
df["ratio_surge_flood"] = (paf.surge/paf.flood).mean(axis=1)


# save

In [106]:
df.to_csv("intermediate/df_for_wrapper.csv")

In [107]:
df.head().T

country,Albania,Angola,Argentina,Armenia,Australia
macro_gdp_pc_pp,9.960941e+03,7.488012e+03,1.808720e+04,7.526836e+03,4.283402e+04
macro_pop,2.897366e+06,2.147162e+07,4.144625e+07,2.976566e+06,2.312930e+07
macro_urbanization_rate,5.740700e-01,4.405000e-01,9.175100e-01,6.267300e-01,8.942300e-01
macro_prepare_scaleup,0.000000e+00,0.000000e+00,7.333333e-01,4.000000e-01,8.000000e-01
macro_borrow_abi,1.750000e-01,1.750000e-01,3.750000e-01,4.250000e-01,9.166667e-01
macro_avg_prod_k,1.852800e-01,4.323974e-01,2.695495e-01,3.666957e-01,2.736024e-01
macro_T_rebuild_K,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00
macro_pi,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01,2.000000e-01
macro_income_elast,1.500000e+00,1.500000e+00,1.500000e+00,1.500000e+00,1.500000e+00
macro_rho,6.000000e-02,6.000000e-02,6.000000e-02,6.000000e-02,6.000000e-02


# testing the result

In [108]:
df=compute_resilience_from_packed_inputs(df)
(df.select_dtypes(exclude=[object]) - compute_resilience_from_packed_inputs(df).select_dtypes(exclude=[object])).abs().max().max()

default_rp detected, droping rp
default_rp detected, droping rp


0.0

In [109]:
(df.risk.sort_values(ascending=False)*100).head()


country
Philippines    6.528659
Honduras       5.996029
Madagascar     5.684972
Peru           5.242277
Lao PDR        4.797752
Name: risk, dtype: float64

# saving

In [110]:
df["iso3"]=pd.read_csv("inputs/names_to_iso.csv", index_col="country").iso3

In [111]:
df.to_csv("intermediate/df_for_wrapper_with_output_and_isocode.csv")

# sorting information

In [112]:
outputs = ["risk", "resilience", "risk_to_assets"]

info = pd.DataFrame(columns=df.columns, index=outputs)
df_with_results=compute_resilience_from_packed_inputs(df)
ref = df_with_results[outputs].dropna()

for v in df:
    df_ = df.dropna().copy()
    try : 
        df_[v] = df[v].mean()
    except TypeError:
        pass
    
    g=compute_resilience_from_packed_inputs(df_)[outputs]
    info[v] = (((g - ref)/ref).abs()).median()

desc =pd.read_csv("inputs/inputs_info_wrapper.csv", index_col="key").descriptor
desc;

info = info.T.sort_values("resilience", ascending=False).rename(index=desc)

default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_rp detected, droping rp
default_

In [113]:
info


,risk,resilience,risk_to_assets
Exposure to riverine and coastal floods (poor),1.763336e-01,1.005534e-01,5.112435e-02
Exposure to riverine and coastal floods (nonpoor),3.163822e-01,8.313003e-02,4.814586e-01
Asset vulnerability for nonpoor people,2.808720e-01,6.864081e-02,3.927796e-01
Wealth of nonpoor people,4.452158e-02,5.876647e-02,1.797036e-02
Wealth of poor people,4.369635e-02,5.116025e-02,2.676887e-02
macro_avg_prod_k,1.852219e-01,5.047139e-02,2.281470e-01
Asset vulnerability for poor people,6.115577e-02,4.770588e-02,1.507327e-02
Total social protection spending,3.554112e-02,3.458294e-02,1.205040e-16
Share of social protection targeted to poor people,3.005598e-02,3.098736e-02,1.295470e-16
Exposure to earthquakes,6.840375e-01,4.260317e-03,7.386762e-01


In [114]:
with pd.ExcelWriter("results/information.xlsx", engine='xlsxwriter') as writer:
    
    workbook=writer.book
    
    info.to_excel(writer,sheet_name="information")
    worksheet=writer.sheets["information"]
    #width of the columns
    worksheet.set_column(0, 0, 20)
    worksheet.set_column(1, 1, 20)
    worksheet.set_column(2, 2, 20)
    worksheet.set_column(3, 3, 20)

    #filter
    worksheet.autofilter('A1:D1')

    #freeze panes
    writer.sheets["information"].freeze_panes(1, 1)

In [115]:
info.sort_values("resilience", ascending=False).head(7)


,risk,resilience,risk_to_assets
Exposure to riverine and coastal floods (poor),0.176334,0.100553,0.051124
Exposure to riverine and coastal floods (nonpoor),0.316382,0.083130,0.481459
Asset vulnerability for nonpoor people,0.280872,0.068641,0.392780
Wealth of nonpoor people,0.044522,0.058766,0.017970
Wealth of poor people,0.043696,0.051160,0.026769
macro_avg_prod_k,0.185222,0.050471,0.228147
Asset vulnerability for poor people,0.061156,0.047706,0.015073


In [116]:
df.ix["Mali"]

macro_gdp_pc_pp                     1589.17
macro_pop                       1.53016e+07
macro_urbanization_rate             0.39916
macro_prepare_scaleup                   0.6
macro_borrow_abi                   0.333333
macro_avg_prod_k                   0.643845
macro_T_rebuild_K                         3
macro_pi                                0.2
macro_income_elast                      1.5
macro_rho                              0.06
macro_shareable                         0.8
macro_max_increased_spending           0.05
macro_fa_glofris                   0.091125
macro_protection                          1
macro_tau_tax                     0.0507364
n_cat_info__nonpoor                     0.8
n_cat_info__poor                        0.2
c_cat_info__nonpoor                 1828.14
c_cat_info__poor                    633.283
axfin_cat_info__nonpoor            0.057306
axfin_cat_info__poor              0.0260487
gamma_SP_cat_info__nonpoor          1.20163
gamma_SP_cat_info__poor         